In [1]:
# dataset: https://www.kaggle.com/c/dogs-vs-cats

# ---------------------
# import required packages
# ---------------------
from __future__ import print_function
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from random import shuffle
from tqdm import tqdm
import torchvision
from sklearn.linear_model import LogisticRegression

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
#Initialziation
IMG_SIZE=28
LR=0.01

In [5]:
#Data reshaping
y_train=y_train.reshape((60000,1))
x_train=x_train.reshape((60000,28,28,1))
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [6]:
# ---------------------
# construct the cnn model for this project
# ---------------------
import tflearn
import tensorflow as tf

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.estimator import regression

def conv_net():
    convnet = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],
                        name = 'input')
    print(convnet)
    # conv_2d(incoming, nb_filter, filter_size, ..., activation)
    convnet = conv_2d(convnet, 32, (3,3), activation = 'relu')
    # max_pool_2d(incoming, kernel_size, ...)
    convnet = max_pool_2d(convnet, 2)
    

    #flatten the layer
    print("break_1")
    flatten = tflearn.flatten(convnet)
    # fully_connected(incoming, n_units, activation, ...)
    convnet = fully_connected(flatten, 5, activation = 'relu')
    # dropout(incoming, keep_prob is drop_prob + keep+prob)

    #standard recommendation for the Net arch    
    logits = fully_connected(convnet, 10, activation = 'softmax')
    #convnet = dropout(convnet, 0.8)
    
    #logits
    #logits = tf.layers.dense(inputs=convnet, units=10)
    
    #regression(incoming, optimizer, learning_rate, loss, name, ...)
    convnet = regression(logits, optimizer = 'adam', learning_rate = LR,
                             loss = 'categorical_crossentropy', name = 'targets')

    return convnet

W0626 23:56:13.431845 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0626 23:56:13.433053 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0626 23:56:13.439254 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0626 23:56:13.444903 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0626 23:56:13.451669 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/

In [7]:
# ---------------------
# define the model
# ---------------------
convnet = conv_net()
model = tflearn.DNN(convnet, tensorboard_dir = 'log')


W0626 23:56:13.461524 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 23:56:13.465680 140735728333696 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0626 23:56:13.467242 140735728333696 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent beha

Tensor("input/X:0", shape=(?, 28, 28, 1), dtype=float32)
break_1


W0626 23:56:13.705239 140735728333696 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0626 23:56:13.947839 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/helpers/trainer.py:134: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



In [8]:
model.fit(x_train,y_train,n_epoch=10)


Training Step: 9379  | total loss: 0.27395 | time: 17.671s
| Adam | epoch: 010 | loss: 0.27395 -- iter: 59968/60000
Training Step: 9380  | total loss: 0.25033 | time: 17.688s
| Adam | epoch: 010 | loss: 0.25033 -- iter: 60000/60000
--


In [9]:
p=model.predict(x_train)

In [10]:
p

array([[7.9721685e-10, 1.6921947e-13, 9.9318043e-10, ..., 1.8597056e-11,
        1.0196896e-04, 1.2178730e-03],
       [9.9857759e-01, 1.1010650e-05, 4.1416576e-04, ..., 1.2678601e-08,
        6.0156407e-04, 3.6364856e-06],
       [4.8358723e-10, 2.8679817e-04, 1.7911589e-05, ..., 3.2650910e-03,
        2.8881966e-04, 3.8544130e-02],
       ...,
       [4.3792874e-17, 7.5091223e-20, 8.8614379e-16, ..., 3.3064304e-16,
        7.8296000e-07, 2.5055010e-04],
       [4.2715464e-02, 8.5906824e-04, 3.7789539e-07, ..., 2.7400571e-09,
        5.7125952e-05, 1.1411099e-05],
       [8.4966337e-07, 1.1174529e-15, 7.2171497e-07, ..., 2.3052129e-15,
        9.9999082e-01, 7.4881950e-06]], dtype=float32)

In [11]:
prediction=np.argmax(p,axis=1)

In [12]:
real_y_label=np.argmax(y_train,axis=1)

In [13]:
train_correct_count=0
for i in range(len(prediction)):
    if (prediction[i]==real_y_label[i]):
        train_correct_count=train_correct_count+1;

In [14]:
print("train Accuracy"+str(train_correct_count/x_train.shape[0]))

train Accuracy0.9784833333333334


In [28]:
test_correct_count=0;
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_test=y_test.reshape((10000,1))
x_test=x_test.reshape((10000,28,28,1))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
p_test=model.predict(x_test);
prediction_test=np.argmax(p_test,axis=1)
for i in range(len(prediction_test)):
    if (prediction_test[i]==y_test[i]):
        test_correct_count=test_correct_count+1;
print("test Accuracy"+str(test_correct_count/x_test.shape[0]))

test Accuracy0.9583


In [29]:
prediction_test

array([7, 2, 1, ..., 4, 5, 6])

In [30]:
y_test

array([[7],
       [2],
       [1],
       ...,
       [4],
       [5],
       [6]], dtype=uint8)

In [ ]:
#Hurray, ;)! 